In [2]:
#Import library yang dibutuhkan
import csv
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
import string

In [ ]:
#Membuka file data tweets
data_tweet = pd.read_csv("file_directory/data.csv", encoding='latin-1')
data_tweet = data_tweet.astype(str)

In [ ]:
#Proses filter
def filtering_text(text):
    # mengubah tweet menjadi huruf kecil
    text = text.lower()
    # menghilangkan url
    text = re.sub(r'https?:\/\/\S+','',text)
    # menghilangkan mention, link, hastag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    #menghilangkan karakter byte (b')
    text = re.sub(r'(b\'{1,2})',"", text)
    # menghilangkan yang bukan huruf
    text = re.sub('[^a-zA-Z]', ' ', text)
    # menghilangkan digit angka
    text = re.sub(r'\d+', '', text)
    #menghilangkan tanda baca
    text = text.translate(str.maketrans("","",string.punctuation))
    # menghilangkan whitespace berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data_tweet['filtered'] = data_tweet['tweet'].apply(filtering_text)
print(data_tweet['filtered'].head())

In [ ]:
#Proses stopwords dan stemming
def stop_stem(text):
    #stopword
    with open('file_directory/kamus.txt') as kamus:
        word = kamus.readlines()
        list_stopword = [line.replace('\n',"") for line in word]
    dictionary = ArrayDictionary(list_stopword)
    stopword = StopWordRemover(dictionary)
    text = stopword.remove(text)
    # stemming
    factory_stemmer = StemmerFactory()
    stemmer = factory_stemmer.create_stemmer()
    text = stemmer.stem(text)
    return text

data_tweet['cleaned'] = data_tweet['filtered'].apply(stop_stem)

print(data_tweet['cleaned'].head())

In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_tweet['tweet_tokens'] = data_tweet['cleaned'].apply(word_tokenize_wrapper)

print(data_tweet['tweet_tokens'].head())

In [ ]:
kamus_normalisasi = pd.read_csv("file_directory/slang.csv")

kata_normalisasi_dict = {}

for index, row in kamus_normalisasi.iterrows():
    if row[0] not in kata_normalisasi_dict:
        kata_normalisasi_dict[row[0]] = row[1] 
        
def normalisasi_kata(document):
    return [kata_normalisasi_dict[term] if term in kata_normalisasi_dict else term for term in document]

data_tweet['normalisasi'] = data_tweet['tweet_tokens'].apply(normalisasi_kata)

data_tweet['normalisasi'].head(10)

In [ ]:
data_tweet.to_csv('file_directory/preprocess.csv', index=False)